In [1]:
%load_ext autoreload
%autoreload 2
import io_backend as io

**Note: this notebook requires `io_backend.py` and `compModel.js` to be in the same directory**

External Merge Sort
------------------

Sorting big data is easy!

In [2]:
import random
x = range(1000000); random.shuffle(x)
%time x.sort()

CPU times: user 690 ms, sys: 10.7 ms, total: 701 ms
Wall time: 840 ms


Except that sorting a million integers isn't cool... you know what's cool?  Sorting a billion!  Try it!

Just kidding! (Hint: if you took the above seriously, select "Kernel >> Restart" in the notebook menu bar and then select "Clear all outputs & restart" to save your computer)

What actually happens when someone- or more pertinently, some DBMS- wants to sort data that is bigger than main memory (e.g. RAM)?  For example, the rows of a very large table in a database?

In this notebook we'll look at one classic algorithm for doing this efficently: **external merge sort**.

Step 1: External Merge
---------------------

First, we'll look at a simple way to efficiently merge a file larger than memory.  Suppose we have the following scenario:
* _Input:_ Two sorted lists of length $N$ and $M$
* _Output:_ One sorted list of length $N+M$

We will see a way to do this that requires at least 3 buffer pages, and is $O(2*(M+N))$:

In [3]:
def min_non_null_idx(a):
    arr = filter(lambda x : x[1] is not None, enumerate(a))
    return min(arr, key=lambda x : x[1])

def external_merge(b, file_in_ids, erase=False):
    P = b.page_size
    if len(file_in_ids) + 1 > b.buffer_size:
        raise Exception("Too many files for buffer size.")

    # A FileIterator will iterate through elements of a file's pages
    # reading in & releasing pages so as to take up one page in buffer
    fis = [io.FileIterator(b, fid) for fid in file_in_ids]
    elements = [fi.get_next() for fi in fis]

    # A FileWriter object will append elements to a file, creating
    # & flushing pages so as to take up one page in buffer
    fid_out = b.new_file()
    fw = io.FileWriter(b, fid_out)
    
    # Successively choose the smallest of the B files' smallest elements
    while any([e is not None for e in elements]):
        
        # Erase & then output the smallest element from the pages currently in buffer
        min_i, min_e = min_non_null_idx(elements)
        if erase:
            fis[min_i].erase_current()
        fw.append(min_e)
        
        # Get the next element from the file we just picked one from
        elements[min_i] = fis[min_i].get_next()
    fw.close()
    return fid_out

In [4]:
def new_rand_file(b, l, sorted=False):
    vals = random.sample(range(10*l), l)
    if sorted:
        vals.sort()
    fid = b.new_file()
    fw = io.FileWriter(b, fid)
    for v in vals:
        fw.append(v)
    fw.close()
    return fid

In [5]:
# Create the buffer + two random sorted files
N = 4
M = 5
b = io.Buffer(page_size=2, buffer_size=3)
fids = [
    new_rand_file(b, N, sorted=True),
    new_rand_file(b, M, sorted=True)
]
b.display_set_mark()  # Don't animate this setup part

# Merge!
merged_fid = external_merge(b, fids, erase=True)

# NOTE THAT YOU CAN ADJUST THE ANIMATION SPEED!!!
b.display(speed=1000)

IO Counts,R,W
To/from Buffer,0,0
To/from Disk,0,0


We see that for two files and three buffer pages, this algorithm is

$O(2*(M+N))$,

since for each page read in from disk ($M+N$ total), there is a page flushed out.  Furthermore, since the algorithm above is fully generalizable to $K$ files and a buffer of size $K+1$ or greater, we see that if file $i$ has $N_i$ pages, then we again are

$O\left(2*\sum_{i=1}^KN_i\right)$

which is just the above in more general form.

External Merge Sort
------------------

Now that we can merge two sorted files, sorting is easy: we just

1. Split our data up into files small enough to _sort in memory_ (i.e. small enough to fit in the buffer)
2. Repeatedly _merge these files_ with the above algorithm until they are again one (now sorted) file!

Since space is an issue for the visualizations, we'll just show a simple sort-merge with one round of merging below, however the algorithm is fully extendable to larger files with more merges:

Helper functions for (1) splitting and sorting in-memory:

In [6]:
import itertools
def flatten(x): return list(itertools.chain.from_iterable(x))

def split_and_sort_in_mem(b, fid_in):
    fids_out = []
    N = b.get_file_len(fid_in)
    B = b.buffer_size
    P = b.page_size
    for i in range(0, N, B):
        fid_out = b.new_file()
        
        # Read in enough pages to fill the buffer if possible
        pages = [b.read(fid_in, pid) for pid in range(i, min(i+B, N))]
        
        # Collect & sort the data from the pages in memory
        vals = sorted(flatten([page.get_data_copy() for page in pages]))
        
        # Write out to the new file
        for j,page in enumerate(pages):
            b.release(page)
            page = b.new_page(fid_out)
            page.set_all(vals[j*P:(j+1)*P])
            b.flush(page)
        fids_out.append(fid_out)
    
    # Delete original file & return new file ids
    b.delete_file(fid_in)
    return fids_out

Helper function for (2) recursively applying external merge:

In [7]:
def recursive_external_merge(b, fids, erase=False):
    L = len(fids)
    if L < b.buffer_size:
        return external_merge(b, fids, erase=erase)
    else:
        mid_1 = recursive_external_merge(b, fids[:(L/2)], erase=erase)
        mid_2 = recursive_external_merge(b, fids[(L/2):], erase=erase)
        return external_merge(b, [mid_1, mid_2], erase=erase)

_The full **external merge-sort** algorithm:_

In [8]:
def external_merge_sort(b, fid, erase=False, concise=False):
    
    # Split into size-B chunks, sort in memory, save to new files
    fids = split_and_sort_in_mem(b, fid)
    
    # Optionally set mark here so as not to animate the previous two steps
    if concise:
        b.display_set_mark()
        
    # Merge the sorted files recursively
    return recursive_external_merge(b, fids, erase=erase)

In [9]:
# Create buffer & random file larger than buffer
b = io.Buffer(buffer_size=3, page_size=2)
fid = new_rand_file(b, 11)
b.display_set_mark()

# Use the algorithm
external_merge_sort(b, fid, erase=True, concise=True)
b.display(buffer_num=1)

IO Counts,R,W
To/from Buffer,0,0
To/from Disk,0,0


What is the cost of this algorithm?

Initial part (splitting and in-memory sorting): $2*N$

Recursive external merge: $\text{ceil}\left(\log_B\left(\frac{N}{B+1}\right)\right)$ passes, each involving $2*N$ IO operations

$\implies O\left(2N*\left(\text{ceil}\left(\log_B\left(\frac{N}{B+1}\right)\right) + 1\right)\right)$